## CONFIG NOTEBOOK

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/ECE_285_proj/
%cd DeepZip_Compression

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/ECE_285_proj
/content/drive/My Drive/ECE_285_proj/DeepZip_Compression


In [2]:
!ls

arithmetic_coder-Copy1.ipynb  data_preparation.py    main.ipynb   README.md
arithmetic_coder.ipynb	      input.txt		     my_model.h5
c_m.npy			      main-checkpoint.ipynb  __pycache__


In [3]:
import sys
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K
np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


## DATA PREP

In [0]:
# use data prep code from: https://github.com/mohit1997/DeepZip
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

In [0]:
bs=128  #batch size
time_steps=64
num_epochs=20

In [98]:
#Generate traning data and its lable 
X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded[: bs * 350],bs, time_steps)  #run with portion of dataset
# X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded,bs, time_steps)  #run with full dataset


(44800, 1)
(44800,)
44672
(44672, 65)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [0]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [0]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

## Codec

In [0]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.high = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold 1/2 max range
        self.low = 0  #initial low 
        self.s = 0

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range = high - low + 1   
        sym = int(sym)
        total = (c[-1])  #max val of cumulative propabilities
        symlow = (c[sym])  #range minimum
        symhigh = (c[sym+1] ) #rage maximum
        

        newlow = low + symlow*range // total  #low in arithmetic integer
        newhigh = low + symhigh*range // total -1 #high in arithemtic integer
        self.low = int(newlow)   
        self.high = int(newhigh)
        range = self.high - self.low 
        
         #renormalization when range is less than half
        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range = range << 1 | 1
            self.high = self.low + range
            self.s = self.s + 1         
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        

class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,bitprecision,bitin,c):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])  
        self.max_range = 1 << bitprecision
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        self.stream = self.input[0:bitprecision]   #inital low is based on first 64 bits of encoded bitstream
        self.low = int(self.stream,2)  #convert low to integer
        self.thresh = []  #initialize threshold
        self.total = (c[-1])  #maximum cumulative probability value for uniform distribution
        self.cum = [c[i]/self.total for i in range(len(c))]  #get cumulative probabilities in decimal form
        self.thresh = [round((self.cum[i])*self.mask) for i in range(len(self.cum))]  #set ranges that correspond to each symbol
        
    def decode(self, c):
        c = [c[i]/self.total for i in range(len(c))]  #for each new distribution, get in decimal form
        #check which range low falls in, decode corresponding symbol 
        for i in range(len(self.thresh)-1):  
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
                rangenew = int(self.thresh[i+1] - self.thresh[i])
                #renormalization 
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
                    self.thresh[i+1] = rangenew + self.thresh[i]
                self.thresh = [(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(self.cum))]  #update ranges
                break
        return sym

## Encoding

In [0]:
#Load model
DZ_model = load_model('my_model.h5', compile=False)

bitprecision = 64
l = int(X.shape[0] / bs) * bs 

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.int64)
c[1:] = np.cumsum(prob*10000000 + 1)        
bitstream = []
enc = ArithmeticEncoder(bitprecision)  #initialize encoder

num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
count =0 


#ENCODING 

#encoding first 128 * 64 symbols with uniform distribution
count = 0
new_l = []
for i in (range(bs)):
    for j in range(min(time_steps, num_iters)):
        if (count%16 == 0 and count != 0):
            enc = None
            enc = ArithmeticEncoder(bitprecision)
            new_l.append(new)
            new = None
        new = enc.write(X[ind[i], j], c.tolist())
        count = count + 1
new_l.append(new)  #append new value to new_l to determine final bitstream 

count = 0
cumul_array = np.zeros(( round((num_iters - time_steps) * bs), 5), dtype = np.uint64)
new_l1 = []
enc1 = ArithmeticEncoder(bitprecision)
cumul = np.zeros((bs, alphabet_size + 1), dtype = np.uint64)
t = 0
for k in (range(num_iters - time_steps)):
    prob = DZ_model.predict(X[ind,:], batch_size=bs)  #use LSTM to predict distribution 
    cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
    enc1 = None
    enc1 = ArithmeticEncoder(bitprecision)
    for i in range(bs):
        new = enc1.write(Y_raw[ind[i]], cumul[i,:].tolist()) 
        count = count + 1
    new_l1.append(new)    
    ind = ind + 1

new = None

In [107]:
##Detremine length of encoded bit stream


bitstream_total = ""
for i in range(len(new_l)):
    count = count + 1
    new = new_l[i]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    
    #add any leading 0s that were truncated, remove any trailing zeros that aren't needed
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    bitstream = bitstream.rstrip('0')
    bitstream_total = bitstream_total + bitstream
# print(len(bitstream_total))
      
bitstream = ""
for i in range(len(new_l1)):
        new = new_l1[i]
        low_final = new[0] #final low value 
        s_final = new[1]  #count of renormalizations
        bitstream = format(int(low_final), 'b') 
        
        #add any leading 0s that were truncated, remove any trailing zeros that aren't needed
        while len(bitstream) < (s_final+bitprecision):
          bitstream = '0' + bitstream
        bitstream = bitstream.rstrip('0')
        bitstream_total = bitstream_total + bitstream

  
print("Bitstream length:",len(bitstream_total))

#calculate total number of bytes
bytes_total = len(bitstream_total)//8
print("Compressed bytes:", bytes_total)

Bitstream length: 136111
Compressed bytes: 17013


##DECODING 

In [0]:
#Decoding first 128 * 64  uniform bits
series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
symdec = []
count = 1
for i in range(len(new_l)):
    count = count + 1
    new = new_l[i]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    dec = None
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    for i in range(16):
        symdec.append(dec.decode(c.tolist()))   
# print(symdec)

In [109]:

# NN decoding for the next (num_iter - time_steps) * bs
symdec_a = np.asarray(symdec)
symdec_a = symdec_a.reshape(128, 64)
series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
series_2d[:,0:64] = symdec_a
bitprecision = 64
l = int(X.shape[0] / bs) * bs 
print(l)
print((X.shape[0])-time_steps)

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.int64)
c[1:] = np.cumsum(prob*10000000 + 1)        
bitstream = []
enc = ArithmeticEncoder(bitprecision)

n_sym = []
count = 0
t = 0
cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)
for j in (range(num_iters - time_steps)):
    prob = DZ_model.predict(series_2d[:,j:j+time_steps], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
    new = new_l1[j]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream 
    c = cumul[0,:]
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    for i in range(bs):       
        c = cumul[i,:]
        n_sym.append(dec.decode(c.tolist()))
        series_2d[i,j+time_steps] = n_sym[-1]
        count = count + 1
        
     

44672
44608


In [0]:
 a = series_2d.reshape(-1)

In [115]:

print("input array:",a[64:350])
print("decoded array:", Y_raw[0:286].T)

input array: [1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1
 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1
 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0
 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2
 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2
 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3
 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1
 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2]
decoded array: [[1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3
  1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3
  1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3
  1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3
  1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3 1 1 0 2 2 3
  1 1 0 2 2 3 1 1 0 2

In [118]:
#Convert data back to DNA sequence
dec_stream = []
for x, sym in enumerate(a): 
    if sym == 0:
        dec_stream.append('a')
    elif sym ==1:
        dec_stream.append('c')
    elif sym == 2:
        dec_stream.append('g')
    elif sym == 3:
        dec_stream.append('t')
        
print("Decoded steam:",dec_stream)

Decoded steam: ['a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c', 'c', 'a', 'g', 'g', 't', 'c',